In [26]:
import sqlite3

connexion = sqlite3.connect('vivino.db')
cursor = connexion.cursor()




In [47]:
query1 = """
SELECT COUNT(wineries.name)
FROM wineries
WHERE EXISTS (
    SELECT 1
    FROM wines
    WHERE wineries.name = wines.name
);

"""
cursor.execute(query1)
cursor.fetchall()

#the number of matches equals to number of wines names and wineries names, therefore, they match

[(1020,)]

In [122]:
#show the wines that have #1 ranking 2 years in row, highest rating given by most people

query2 = """
SELECT DISTINCT wines.name, wines.ratings_average, wines.ratings_count, wines.winery_id
FROM wines
JOIN vintages ON wines.id = vintages.wine_id
JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
JOIN toplists ON vintage_toplists_rankings.top_list_id = toplists.id
WHERE vintage_toplists_rankings.rank = 1 AND vintage_toplists_rankings.previous_rank = 1
GROUP BY toplists.name
ORDER BY wines.ratings_average DESC, wines.ratings_count DESC
LIMIT 3
"""
cursor.execute(query2)
result = cursor.fetchall()

for i in range(len(result)):

    print(f"The {str(i+1)} degree award goes to winery number {str(result[i][3])}, for their {result[i][0]}, rated {str(result[i][1])} by {str(result[i][2])} users!")

[('Cabernet Sauvignon', 4.8, 2941, 14919), ('Toscana', 4.7, 16284, 82979), ('Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 4.7, 9564, 3286)]
The 1 degree award goes to winery number 14919, for their Cabernet Sauvignon, rated 4.8 by 2941 users!
The 2 degree award goes to winery number 82979, for their Toscana, rated 4.7 by 16284 users!
The 3 degree award goes to winery number 3286, for their Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne, rated 4.7 by 9564 users!


In [89]:
#show the wines that have ranking 1 

# guaranteed_rating is the 95% percentile of the rating, based on the sample size
# guaranteed_rating := rating - 1.96 * sqrt(rating*(1-rating)/count)
# https://www.omnicalculator.com/statistics/sampling-error

query2 = """
SELECT DISTINCT wines.name, wines.ratings_average, wines.ratings_count, 5.0*(((wines.ratings_average/5.0) - 1.96 * SQRT((wines.ratings_average/5.0)*(1.0-(wines.ratings_average/5.0))/wines.ratings_count))) AS guaranteed_rating
FROM wines
JOIN vintages ON wines.id = vintages.wine_id
JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
JOIN toplists ON vintage_toplists_rankings.top_list_id = toplists.id
WHERE vintage_toplists_rankings.rank = 1 AND vintage_toplists_rankings.previous_rank = 1
GROUP BY toplists.name
ORDER BY guaranteed_rating DESC
LIMIT 3
"""

cursor.execute(query2)
result = cursor.fetchall()
print(result)
wine_names = []
for i in result:
    wine_names.append(i[0])

[('Cabernet Sauvignon', 4.8, 2941, 4.76458851205599), ('Toscana', 4.7, 16284, 4.6817616695226825), ('Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 4.7, 9564, 4.676201704983944)]
['Cabernet Sauvignon', 'Toscana', 'Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne']


In [107]:
for w in wine_names:
    query = f""" SELECT name, id FROM wineries WHERE wineries.name = '{w}'"""
    cursor.execute(query)
    result = cursor.fetchall()
    print(result)

#Turns out wineries can make the same wine

winery_ids = [14919, 82979, 3286]

for w in winery_ids:
    query = f""" SELECT name, id FROM wineries WHERE wineries.id = '{w}'"""
    cursor.execute(query)
    result = cursor.fetchall()
    print(result)

# winery_ids from the wines table do not exist in wineries table... 

[('Cabernet Sauvignon', 16578), ('Cabernet Sauvignon', 61274), ('Cabernet Sauvignon', 66284), ('Cabernet Sauvignon', 94912), ('Cabernet Sauvignon', 1144227), ('Cabernet Sauvignon', 1145391), ('Cabernet Sauvignon', 1611255), ('Cabernet Sauvignon', 2816852)]
[('Toscana', 1155217), ('Toscana', 1171108), ('Toscana', 2446729)]
[('Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 1127349)]
[]
[]
[]


In [70]:
query3 = """
SELECT DISTINCT name
FROM toplists
"""
cursor.execute(query3)
result = cursor.fetchall()
awards = []
for i in range(len(result)):
    awards.append(result[i][0])

print (awards)



["Vivino's 2016 Wine Style Awards: Italian Barbaresco", "Vivino's 2016 Wine Style Awards: Italian Barolo", "Vivino's 2016 Wine Style Awards: Italian Bolgheri", "Vivino's 2017 Wine Style Awards: Italian Amarone", "Vivino's 2017 Wine Style Awards: Italian Barolo", "Vivino's 2017 Wine Style Awards: Italian Bolgheri", "Vivino's 2018 Wine Style Awards: Italian Amarone", "Vivino's 2018 Wine Style Awards: Italian Barbaresco", "Vivino's 2018 Wine Style Awards: Italian Barolo", "Vivino's 2018 Wine Style Awards: Italian Bolgheri", "Vivino's 2019 Wine Style Awards: Italian Barbaresco", "Vivino's 2019 Wine Style Awards: Italian Barolo", "Vivino's 2019 Wine Style Awards: Italian Bolgheri", "Vivino's 2020 Wine Style Awards: Italian Amarone", "Vivino's 2020 Wine Style Awards: Italian Barbaresco", "Vivino's 2020 Wine Style Awards: Italian Barolo", "Vivino's 2020 Wine Style Awards: Italian Bolgheri", "Vivino's 2016 Wine Style Awards: Italian Brunello", "Vivino's 2016 Wine Style Awards: Central Italy Re

In [93]:
query4 = """ 
SELECT w.name, w.id, 
wines.name, wines.ratings_average, wines.ratings_count, 5.0*(((wines.ratings_average/5.0) - 1.96 * SQRT((wines.ratings_average/5.0)*(1.0-(wines.ratings_average/5.0))/wines.ratings_count))) AS guaranteed_rating
FROM wineries w
JOIN wines ON wines.name = w.name
JOIN vintages ON wines.id = vintages.wine_id
JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
JOIN toplists ON vintage_toplists_rankings.top_list_id = toplists.id
WHERE vintage_toplists_rankings.rank = 1 AND vintage_toplists_rankings.previous_rank = 1
GROUP BY toplists.name
ORDER BY guaranteed_rating DESC
"""

cursor.execute(query4)
result = cursor.fetchall()
for row in result:
    print(row)

('Cabernet Sauvignon', 16578, 'Cabernet Sauvignon', 4.8, 2941, 4.76458851205599)
('Toscana', 1155217, 'Toscana', 4.7, 16284, 4.6817616695226825)
('Toscana', 1155217, 'Toscana', 4.7, 16284, 4.6817616695226825)
('Toscana', 1155217, 'Toscana', 4.7, 16284, 4.6817616695226825)
('Toscana', 1155217, 'Toscana', 4.7, 16284, 4.6817616695226825)
('Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 1127349, 'Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne', 4.7, 9564, 4.676201704983944)
('Châteauneuf-du-Pape', 18017, 'Châteauneuf-du-Pape', 4.7, 7544, 4.673204319684638)
('Châteauneuf-du-Pape', 18017, 'Châteauneuf-du-Pape', 4.7, 7544, 4.673204319684638)
('Châteauneuf-du-Pape', 18017, 'Châteauneuf-du-Pape', 4.7, 7544, 4.673204319684638)
('Châteauneuf-du-Pape', 18017, 'Châteauneuf-du-Pape', 4.7, 7544, 4.673204319684638)
('Bolgheri Superiore', 22447, 'Bolgheri Superiore', 4.6, 47939, 4.5878571537077715)
('Château Margaux (Premier Grand Cru Classé)', 1127795, 'Château Margaux (Premier Grand Cru Cla